<a href="https://colab.research.google.com/github/MOOwuttichai/Cancer_Social_Listening/blob/main/%E0%B9%80%E0%B9%80%E0%B8%A2%E0%B8%81%E0%B8%A1%E0%B8%B0%E0%B9%80%E0%B8%A3%E0%B9%87%E0%B8%87%2B%E0%B9%80%E0%B8%9E%E0%B8%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
!pip install attacut
!pip install pythainlp
!pip install epitran

ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 467, in run
    env = get_environment(lib_locations)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/metadata/__init__.py", line 85, in get_environment
    return select_backend().Environment.from_paths(paths)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/metadata/pkg_resources.py", line 232, in from_paths
    return cls(pkg_resources.WorkingSet(paths))
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 613, in __init__
    self.add_entry(entr

In [148]:
import pandas as pd
from google.colab import drive
from attacut import tokenize, Tokenizer
import pythainlp
from pythainlp import sent_tokenize, word_tokenize, Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from collections import OrderedDict
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [149]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/ดึงข้อความFacbook - Sheet1.csv')

In [150]:
data.describe()

,ลำดับที่
count,1374.000000
mean,687.500000
std,396.783946
min,1.000000
25%,344.250000
50%,687.500000
75%,1030.750000
max,1374.000000


In [151]:
data.head(2)

,ลำดับที่,ชื่อ,คำพูดโรค
0,1,Wiraon Chuenchom,ตระกูลทางผมก็มะเร็งทั้งย่า อา แต่ที่เคยคุยกับค...
1,2,Wiraon Chuenchom,จากนั้น กค. จะมีก้อนโผ่ลมา และมีการปวดเลยไปหาห...


In [152]:
data.iloc[0,1]

'Wiraon Chuenchom'

In [153]:
data[data['ชื่อ']==data.iloc[1,1]]

,ลำดับที่,ชื่อ,คำพูดโรค
0,1,Wiraon Chuenchom,ตระกูลทางผมก็มะเร็งทั้งย่า อา แต่ที่เคยคุยกับค...
1,2,Wiraon Chuenchom,จากนั้น กค. จะมีก้อนโผ่ลมา และมีการปวดเลยไปหาห...


In [154]:
comment=data.groupby('ชื่อ').sum().reset_index()

In [155]:
comment[comment['ชื่อ']=='Wiraon Chuenchom']

,ชื่อ,ลำดับที่,คำพูดโรค
839,Wiraon Chuenchom,3,ตระกูลทางผมก็มะเร็งทั้งย่า อา แต่ที่เคยคุยกับค...


In [207]:
text = comment['คำพูดโรค'][1]

In [208]:
text

'ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรัสตับอักเสบซี และมีตับแข็ง ใช้สิทธิ์เบิกได้ข้าราชการ รักษาที่รพ.ศิริราช ทานยาครบ3เดือน จนค่าไวรัสอยู่ในเกณฑ์ปกติ F/U + U/S ทุกๆ6เดือน จนช่วงโควิด Loss F/U ไปอีกที ปี64 U/S พบ HCC stage3 ได้ทำ TACE 5 ครั้ง + RFA 2 ครั้ง จนหายปกติ ตอนนี้ มีนัดทุก6เดือนติดตามอาการคะ'

In [209]:
words = set(thai_words())
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
  words.add(i)
custom_tokenizer = Tokenizer(words)
Token = custom_tokenizer.word_tokenize(normalize(text))

In [210]:
Token

['ปี',
 '62',
 ' ',
 'แม่',
 'เกร็ด',
 'เลือด',
 'ต่ำ',
 '+',
 'ค่า',
 'ตับ',
 'ผิดปกติ',
 ' ',
 'ตรวจ',
 'พบ',
 'ไวรัส',
 'ตับอักเสบ',
 'ซี',
 ' ',
 'และ',
 'มี',
 'ตับแข็ง',
 ' ',
 'ใช้',
 'สิทธิ์',
 'เบิก',
 'ได้',
 'ข้าราชการ',
 ' ',
 'รักษา',
 'ที่',
 'รพ.',
 'ศิริราช',
 ' ',
 'ทาน',
 'ยา',
 'ครบ',
 '3',
 'เดือน',
 ' ',
 'จน',
 'ค่า',
 'ไวรัส',
 'อยู่ในเกณฑ์',
 'ปกติ',
 ' ',
 'F',
 '/U',
 ' ',
 '+',
 ' ',
 'U',
 '/S',
 ' ',
 'ทุกๆ',
 '6',
 'เดือน',
 ' ',
 'จน',
 'ช่วง',
 'โควิด',
 ' ',
 'Loss',
 ' ',
 'F',
 '/U',
 ' ',
 'ไป',
 'อีกที',
 ' ',
 'ปี',
 '64',
 ' ',
 'U',
 '/S',
 ' ',
 'พบ',
 ' ',
 'HCC',
 ' ',
 'stage',
 '3',
 ' ',
 'ได้',
 'ทำ',
 ' ',
 'TACE',
 ' ',
 '5',
 ' ',
 'ครั้ง',
 ' ',
 '+',
 ' ',
 'RFA',
 ' ',
 '2',
 ' ',
 'ครั้ง',
 ' ',
 'จน',
 'หาย',
 'ปกติ',
 ' ',
 'ตอนนี้',
 ' ',
 'มีนัด',
 'ทุก',
 '6',
 'เดือน',
 'ติดตาม',
 'อาการ',
 'คะ']

In [211]:
list_cancer = []
for i in range(len(Token)):
  if (Token[i] == "มะเร็ง")|(Token[i] == "โรคมะเร็ง"):
    list_cancer.append(Token[i]+Token[i+1])
  else :
    list_cancer.append('ไม่สามารถระบุได้')
unique_list = list(OrderedDict.fromkeys(list_cancer)) # ลบคำซ้ำ

In [212]:
unique_list

['ไม่สามารถระบุได้']

In [213]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปากมดลูก','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งเม็ดเลือดขาว','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งท่อน้ำดี']

In [214]:
list_define_cancer = []
for i in range(len(unique_list)):
  for j in range(len(name_cancar)):
    if unique_list[i]==name_cancar[j]:
      list_define_cancer.append(unique_list[i])
if list_define_cancer==[]:
  list_define_cancer.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')

In [215]:
list_define_cancer

['ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น']

#เเบ่งเพศ

In [224]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ','ปู่ทวด','ตาทวด','ลุง','อา','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย','พ่อตา']
          ,'หญิง':['แม่','เเม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า่','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้']}

In [226]:
genden_list =[]
list_genden = []
for i in range(len(Token)):
  for j in range(len(Genden['หญิง'])):
    if (Token[i] == Genden['ชาย'][j]):
      list_genden.append('เพศชาย')
    elif(Token[i] == Genden['หญิง'][j]):
      list_genden.append('เพศหญิง')
genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ

In [227]:
genden_list

['เพศหญิง']

In [169]:
list_define_genden = []
for i in range(len(genden_list)):
  if len(genden_list) == 2:
    list_define_genden.append('เล่าทั้งสองเพศ')
  elif list_define_cancer==[]:
    list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  else:
    list_define_genden.append(genden_list[0])

In [170]:
list_define_genden

['เพศชาย']

#ลองทั้งหมด

In [171]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/ดึงข้อความFacbook - Sheet1.csv')

In [172]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปากมดลูก','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งเม็ดเลือดขาว','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งลำไส้'
,'มะเร็งท่อน้ำดี']

In [228]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ','ปู่ทวด','ตาทวด','ลุง','อา','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย','พ่อตา']
          ,'หญิง':['แม่','เเม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า่','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้']}

In [174]:
words = set(thai_words())
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร']
for i in name:
  words.add(i)

In [176]:
new_colcan = []
new_colgenden = []
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  #หาโรค
  #--------------------------------------------------------
  list_cancer = []
  for i in range(len(Token)):
    if (Token[i] == "มะเร็ง")|(Token[i] == "โรคมะเร็ง"):
      list_cancer.append(Token[i]+Token[i+1])
    # else :
    #   list_cancer.append('ไม่สามารถระบุได้')
  unique_list = []
  unique_list = list(OrderedDict.fromkeys(list_cancer))
  #----------------------------------------------------------
  list_define_cancer = []
  for i in range(len(unique_list)):
    for j in range(len(name_cancar)):
      if unique_list[i]==name_cancar[j]:
        list_define_cancer.append(unique_list[i])
  if list_define_cancer==[]:
    list_define_cancer.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  new_colcan.append(list_define_cancer)


In [180]:
 #หาเพศ
  #----------------------------------------------------------
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  list_genden = []
  for i in range(len(Token)):
    for j in range(len(Genden['หญิง'])):
      if (Token[i] == Genden['ชาย'][j]):
        list_genden.append('เพศชาย')
      elif(Token[i] == Genden['หญิง'][j]):
        list_genden.append('เพศหญิง')
  genden_list =[]
  genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ
  #-------------------------------------------------------------------
  list_define_genden = []
  for i in range(len(genden_list)):
    if len(genden_list) == 2:
      list_define_genden.append('เล่าทั้งสองเพศ')
    elif list_define_cancer==[]:
      list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
    else:
      list_define_genden.append(genden_list[0])
    genden_list_de =[]
    genden_list_de = list(OrderedDict.fromkeys(list_define_genden))
    new_colgenden.append(genden_list_de)

In [195]:
new_colgenden

[['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศหญิง'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เพศหญิง'],
 ['เพศหญิง'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เล่าทั้งสองเพศ', 'เล่าทั้งสองเพศ'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศหญิง'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศชาย'],
 ['เพศหญิง'],
 ['เพศชาย'],
 ['เพศช

In [189]:
genden=[]
for i in range(len(new_colgenden)):
  x= set(new_colgenden[i])
  genden.append(x)

In [229]:
use_data = comment[['ชื่อ','คำพูดโรค']]
use_data['ประเภทของมะเร็ง'] = new_colcan
use_data

,ชื่อ,คำพูดโรค,ประเภทของมะเร็ง
0,\r\nAtsanai Nenraksa,ไม่ครับ เขาคลำตรงคอดูครับแล้วให้เรากลืนน้ำลายค...,[ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น]
1,A Achi Sarakam,ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรั...,[ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น]
2,A Ew Rattikhan Saykammul,แม่เป็นมะเร็งในเม็ดเลือด แต่มีก้อนที่กระเพาะอา...,[ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น]
3,AI'Jack TheRam,ท้ายพอไปตรวจtcสแกน มะเร็งก็ได้ลามไปกระดูกสันหล...,[ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น]
4,Abdfbaba Chillchill,แม่เป็นมะเร็งปอดค่ะ\r\nเมื่อมิ.ย.62มีอาการแรกเ...,[มะเร็งปอด]
...,...,...,...
1033,ไซมึ้ง ชวยเสาะ,ถ่าย 3 4 ครั้ง นี่เหลว หรือ แข็ง ครับขอถามอีกอ...,[ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น]
1034,ไพบูลย์ จณะวัตร,อาการมะเร็งรังไข่ เริ่มต้น คือ ท้องอืด เป็นประ...,[มะเร็งรังไข่]
1035,ไพรัตน์ ชอบกินข้าวเช้า,แม่ผมเป็นมะเร็งเม็ดเลือดขาวเฉียบพลันชนิดมัยอิล...,"[มะเร็งเม็ดเลือดขาว, มะเร็งเม็ดเลือดขาว]"
1036,ไม่รู้ ไม่ชี้,ก่อนกลืนแร่ ผ่าตัดออก ยังไม่พบการลุกลามไปต่อมน...,[ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น]
